In [1]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
# read from melodyData.txt
with open('out/melodyData.txt', 'r') as f:
    parts = eval(f.read())
    print ("Number of parts:" + str(len(parts)))

# one-hot encoding

encodings = {}
encodingIndex = 0
for part in parts:
    for note in part:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1

decodings = {}
for k, v in encodings.items():
    decodings[v] = k        

# encode everything in a

data_encoded = []
for part in parts:
    data_encoded.append([encodings[note] for note in part])


Number of parts:5


In [49]:
num_data = min(200, len(data_encoded))

# X_train should contain sequences from parts 1-100
# X_test should contain the answers to said sequences
# Y_train should contains sequences from parts 201-210
# Y_test should contain the answers to said sequences

def generate_data(data_encoded, encodings):
    X = []
    Y = []
    # given data_encoded, generate training data by looping 
    for i in range(len(data_encoded[:100])):
        currentX = []
        currentY = []
        for j in range(len(data_encoded[i])-1):
            currentX.append(data_encoded[i][j])
            currentY.append(data_encoded[i][j+1])
        X.append(currentX)
        Y.append(currentY)

    X_onehot = []
    for seq in X:
        onehot = np.zeros((len(seq), len(encodings)))
        for note_index in range(len(seq)):
            onehot[note_index][seq[note_index]] = 1
        X_onehot.append(onehot)
    X = X_onehot

    Y_onehot = []
    for seq in Y:
        onehot = np.zeros((len(seq), len(encodings)))
        for note_index in range(len(seq)):
            onehot[note_index][seq[note_index]] = 1
        Y_onehot.append(onehot)
    Y = Y_onehot

    return X, Y

import math

X_train, Y_train = generate_data(data_encoded[:math.floor(num_data*0.7)], encodings)
X_test, Y_test = generate_data(data_encoded[math.floor(num_data*0.7):], encodings)


In [69]:
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(10, stateful=True, batch_input_shape=(1, None, len(encodings))),  
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='softplus'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(encodings), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model one time step at a time
for epoch in range(10):
    print(f'Epoch {epoch + 1}/10')
    for i, sequence in enumerate(X_train):
        model.reset_states()  # Reset states at the beginning of each sequence
        
        # Train on single time step
        for t in range(sequence.shape[0]): 
            x = sequence[t:t+1].reshape(1, 1, len(encodings))
            y = Y_train[i][t:t+1] 
            loss = model.train_on_batch(x, y)


Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [63]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_13 (SimpleRNN)   (1, 10)                   160       
                                                                 
 dense_65 (Dense)            (1, 64)                   704       
                                                                 
 dense_66 (Dense)            (1, 64)                   4160      
                                                                 
 dense_67 (Dense)            (1, 128)                  8320      
                                                                 
 dense_68 (Dense)            (1, 128)                  16512     
                                                                 
 dense_69 (Dense)            (1, 5)                    645       
                                                                 
Total params: 30501 (119.14 KB)
Trainable params: 305

In [67]:
import copy
# given a sequence, generate the next note (up to ten times)

model.reset_states()

# generate the first 10 notes after we input 1

#pred = model.predict(encodings[1])
one_hot = np.zeros((len(encodings)))
one_hot[1] = 1
print (np.array([one_hot]))
pred = model.predict(np.array([[one_hot]]))
for i in range(10):
    one_hot = np.zeros((len(encodings)))
    one_hot[np.argmax(pred)] = 1
    pred = model.predict(np.array([[one_hot]]))
    print (decodings[np.argmax(pred)])


[[0. 1. 0. 0. 0.]]
1/1 [==============================] - 0s 26ms/step
3
1/1 [==============================] - 0s 30ms/step
1
1/1 [==============================] - 0s 28ms/step
2
1/1 [==============================] - 0s 27ms/step
3
1/1 [==============================] - 0s 27ms/step
1
1/1 [==============================] - 0s 33ms/step
2
1/1 [==============================] - 0s 30ms/step
3
1/1 [==============================] - 0s 28ms/step
1
1/1 [==============================] - 0s 28ms/step
2
1/1 [==============================] - 0s 27ms/step
3


'\nfor i in range(10):\n    pred = model.predict(np.array([seq]))\n    pred = np.argmax(pred)\n    print (decodings[pred])\n    # append to sequence the one-hot encoding of pred\n    one_hot = np.zeros((len(encodings)))\n    one_hot[pred] = 1\n    total_seq = np.append(total_seq, [one_hot], axis=0)\n    # set sequence to be the last ten values of total_seq\n    seq = total_seq[-10:]\n\n    #total_seq.append(pred)\n    # total seq is a numpy array, can\'t just append things to it\n\nmapping = {0: "C", 1: "C#", 2: "D", 3: "D#", 4: "E", 5: "F", 6: "F#", 7: "G", 8: "G#", 9: "A", 10: "A#", 11: "B"}\n'

In [ ]:
# given a sequence, generate the next note (up to ten times)
seq = copy.deepcopy(X_test[4])
total_seq = seq[:]
for i in range(10):
    pred = model.predict(np.array([seq]))
    pred = np.argmax(pred)
    # append to sequence the one-hot encoding of pred
    one_hot = np.zeros((len(encodings)))
    one_hot[pred] = 1
    total_seq = np.append(total_seq, [one_hot], axis=0)
    # set sequence to be the last ten values of total_seq
    seq = total_seq[-10:]

    #total_seq.append(pred)
    # total seq is a numpy array, can't just append things to it

mapping = {0: "C", 1: "C#", 2: "D", 3: "D#", 4: "E", 5: "F", 6: "F#", 7: "G", 8: "G#", 9: "A", 10: "A#", 11: "B"}
    

for note in total_seq:
    pitch, dur = decodings[np.argmax(note)]
    print (mapping[pitch % 12], dur)

# convert numbers to musical note letters
# 60 = C

In [2]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split